In [ ]:
import onnx
import os
import wandb
import onnxruntime as ort
import numpy as np
from utils.utils import *
from test import test as evals


## Loading the model

In [ ]:

model = onnx.load(os.getcwd()+"/model.onnx")
(onnx.checker.check_model(model))

## See how the model is saved

In [ ]:
print(model)

## Recover Data as we had in the training to get the test

In [ ]:
train, test = get_data(train=True), get_data(train=False)
test_loader = make_loader(test, batch_size=128)

## Using  the functionalitis fof the onnx

In [ ]:
print(test_loader.dataset.dataset.data.float().unsqueeze(0).numpy().shape)

In [ ]:
ort_sess = ort.InferenceSession(os.getcwd()+"/model.onnx")
ort_input = {ort_sess.get_inputs()[0].name: test_loader.dataset.dataset.data.float().unsqueeze(1).numpy()} # The unsqueeze because the session is expecting a batch dimension

# Making the run session
outputs = ort_sess.run(None, ort_input)[-1]
predicted, actual = outputs.argmax(1), test_loader.dataset.dataset.targets.numpy()


In [ ]:
print("The accuracy is: ",np.sum(predicted == actual)/actual.shape[0])
